In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras import callbacks
import keras
import keras_tuner
import keras.utils as image
from keras import layers
from keras import ops
from keras import callbacks

## Importing Data from csv files


In [2]:
images = pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/images.txt", sep=r'\s+', names=['image_id', 'image_name'], engine='python')
train_test_split = pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/train_test_split.txt", sep=r'\s+', names=['image_id', 'is_training_image'], engine='python')
classes =pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/classes.txt", sep=r'\s+', names=['class_id', 'class_name'], engine='python')
image_class_labels =pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/image_class_labels.txt", sep=r'\s+', names=['image_id', 'class_id'], engine='python')

## Preprocessing

In [3]:
# Merge dfs based on column names so we have one df with all the necessary info contained per each row
image_data = pd.merge(images,train_test_split, on='image_id')
image_data = pd.merge(image_data,image_class_labels, on='image_id')
image_data = pd.merge(image_data,classes, on='class_id')

In [4]:
# Split training and testing image data
training_image_data = image_data[image_data['is_training_image']==1]
testing_image_data = image_data[image_data['is_training_image']==0]

# Shuffle training data
training_image_data = training_image_data.sample(frac=1)

# Initiate empty lists for training and testing images
training_images = []
testing_images = []

# Add training and testing images to corresponding lists
for i in (training_image_data['image_name'].values):
    training_images.append(image.load_img('/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/images/{}'.format(i), target_size=(224, 224)))

for i in (testing_image_data['image_name'].values):
    testing_images.append(image.load_img('/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/images/{}'.format(i), target_size=(224, 224)))

# Extract class labels for training and testing images
training_class_label = np.array(training_image_data['class_id'].values)
testing_class_label = np.array(testing_image_data['class_id'].values)

In [5]:
# Convert list of images to NumPy array
training_images = np.array(training_images)
testing_images = np.array(testing_images)

# Apply preprocessing
preprocessed_training_images = preprocess_input(training_images)
preprocessed_testing_images = preprocess_input(testing_images)

We begin training by keeping the existing layers frozen, allowing only the newly added classification head to learn. This ensures that the output layer is properly trained before fine-tuning the deeper layers. Once the classifier stabilizes, we gradually unfreeze and fine-tune the earlier layers, optimizing them in a controlled manner. This approach prevents instability in feature extraction and allows each layer to adjust effectively, improving overall model performance.

In [11]:
# Load pre-trained Xception model (without top layers)
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)

# Add new layers on top
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
dropout = keras.layers.Dropout(0.5, noise_shape=None, seed=None)(avg) 
output = keras.layers.Dense(201, activation="softmax")(dropout)  # 200 classes
model = keras.Model(inputs=base_model.input, outputs=output)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Learning rate schedule
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,  # Lower LR for stability
    decay_steps=10000,
    decay_rate=0.9
)

# Define optimizer
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

# Compile model
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Set early stopping
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=3, restore_best_weights=True)

# Define validation data split at 30%
split_idx = int(len(preprocessed_training_images) * 0.7)
X_train, X_val = preprocessed_training_images[:split_idx], preprocessed_training_images[split_idx:]
y_train, y_val = training_class_label[:split_idx], training_class_label[split_idx:]

y_train = np.array(y_train).astype(np.int32)
y_val = np.array(y_val).astype(np.int32)

print(len(X_train), len(y_train))
print(len(X_val), len(y_val))
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
# Train model
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val), batch_size=32, callbacks=[earlystopping])

4195 4195
1799 1799
(4195, 224, 224, 3) (4195,)
(1799, 224, 224, 3) (1799,)
Epoch 1/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 138s 1s/step - accuracy: 0.0085 - loss: 5.3418 - val_accuracy: 0.0311 - val_loss: 5.1121
Epoch 2/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - accuracy: 0.0517 - loss: 5.0173 - val_accuracy: 0.0856 - val_loss: 4.8626
Epoch 3/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - accuracy: 0.1282 - loss: 4.7261 - val_accuracy: 0.1334 - val_loss: 4.6407
Epoch 4/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.1763 - loss: 4.4809 - val_accuracy: 0.1579 - val_loss: 4.4322
Epoch 5/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.2471 - loss: 4.2336 - val_accuracy: 0.1879 - val_loss: 4.2488
Epoch 6/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.2759 - loss: 4.0322 - val_accuracy: 0.2123 - val_loss: 4.0803
Epoch 7/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - accuracy: 0.3264 - loss: 3.8451 - val_accuracy: 0.2368 - val_loss: 3.9284
Epoch 8/30
132/132 ━━

In [ ]:
model.evaluate(preprocessed_testing_images[0], preprocessed_testing_images[1])

182/182 ━━━━━━━━━━━━━━━━━━━━ 1003s 5s/step - accuracy: 0.6479 - loss: 1.9867


[4.9380340576171875, 0.42716604471206665]

In [ ]:
keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=None,
    shear_range=0.2,
    zoom_range=0.3,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=Preprocess(),
    validation_split=0.4,
)

'''img = load_img('data/train/cats/cat.0.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
'''
# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely